In [1]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))


from src.config.setup import SetupConfig
from src.data_workflow.create_tables.create_tables import retrieve_all_table_data

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read View
view_di = retrieve_all_table_data('view_data_integration')
view_di.head()

,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA_ACC,HORA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,FECHA_HORA_ACC,DIRECCION,GRAVEDAD,...,ESTADO,FECHA_NACIMIENTO,EDAD,GENERO,MUERTE_POSTERIOR,FECHA_POSTERIOR_MUERTE,CLASE_DE_VEHICULO,TIPO_DE_SERVICIO,MODALIDAD,ENFUGA
0,A000033958,514216,2014-10-13 19:00:00,14:40:00,2014,OCTUBRE,MARTES,2014-10-14 09:40:00,CL 17-KR 114 02,SOLO DANOS,...,ILESO,1989-05-09 19:00:00,25.0,MASCULINO,N,1975-01-01 01:00:00,"CAMION, FURGON",PUBLICO,CARGA-PESADA,N
1,A000037228,4399672,2014-11-27 19:00:00,15:20:00,2014,NOVIEMBRE,VIERNES,2014-11-28 10:20:00,AK 72-CL 13 02,SOLO DANOS,...,ILESO,1966-06-01 19:00:00,48.0,MASCULINO,N,1975-01-01 01:00:00,TRACTOCAMION,PUBLICO,CARGA-PESADA,N
2,A000036014,4399723,2014-11-27 19:00:00,05:10:00,2014,NOVIEMBRE,VIERNES,2014-11-28 00:10:00,AV AVENIDA BOYACA-CL 61 S 42,SOLO DANOS,...,ILESO,1979-02-04 19:00:00,35.0,MASCULINO,N,1975-01-01 01:00:00,VOLQUETA,PARTICULAR,None,N
3,726324600,26010,2007-05-14 19:00:00,15:10:00,2007,MAYO,MARTES,2007-05-15 10:10:00,CL 80-69B 7,SOLO DANOS,...,ILESO,1938-03-08 19:00:00,69.0,MASCULINO,N,1975-01-01 01:00:00,AUTOMOVIL,PARTICULAR,None,None
4,712751700,21517,2007-03-30 19:00:00,13:40:00,2007,MARZO,SABADO,2007-03-31 08:40:00,AV EL DORADO-KR 82 70,SOLO DANOS,...,ILESO,1968-02-09 19:00:00,39.0,MASCULINO,N,1975-01-01 01:00:00,AUTOMOVIL,None,None,None


In [3]:
# view_di.columns
view_di.groupby(["ESTADO"]).size()

ESTADO
HERIDO    250680
ILESO     757372
MUERTO      8141
dtype: int64

In [4]:
# Función para convertir horas y minutos en float
def time_to_float(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    total_minutes = hours * 60 + minutes
    float_value = total_minutes / 60.0
    return float_value


def float_to_time(float_value):
    hours = int(float_value)
    minutes = int((float_value - hours) * 60)
    time_str = "{:02d}:{:02d}:00".format(hours, minutes)
    return time_str

def clasificar_vehiculo(clase_vehiculo):
    DOS_LLANTAS = ['BICICLETA', 'BICITAXI', 'CUATRIMOTO', 'MOTOCARRO', 'MOTOCICLETA', 'MOTOCICLO', 'TRACCION ANIMAL']
    CUATRO_LLANTAS = ['AMBULACIA', 'AUTOMOVIL', 'BUS', 'BUS ALIMENTADOR', 'BUS ARTICULADO', 'BUSETA',
                      'CAMION, FURGON', 'CAMIONETA', 'CAMPERO', 'M. AGRICOLA', 'M. INDUSTRIAL', 'METRO',
                      'MICROBUS', 'NO IDENTIFICADO', 'OTRO', 'REMOLQUE', 'TRACTOCAMION', 'TREN', 'VOLQUETA']
    if clase_vehiculo in DOS_LLANTAS:
        return 'DOS_LLANTAS'
    elif clase_vehiculo in CUATRO_LLANTAS:
        return 'CUATRO_LLANTAS'
    else:
        return 'NO_CLASIFICADO'




In [19]:
# view_di.groupby(["CLASE_DE_VEHICULO"]).size()

In [5]:
INTERES = ["HORA_OCURRENCIA_ACC", "MES_OCURRENCIA_ACC", "DIA_OCURRENCIA_ACC" , "GRAVEDAD", "LOCALIDAD", "CLASE_DE_VEHICULO", "CONDICION_ACCIDENTADO", "nroinfracciones", "TIPO_DE_SERVICIO", "GENERO", "ESTADO"]
data = view_di[INTERES]
data['HORA_OCURRENCIA_ACC_FLOAT'] = data['HORA_OCURRENCIA_ACC'].apply(time_to_float)
data['CLASIFICACION_VEHICULO'] = data['CLASE_DE_VEHICULO'].apply(clasificar_vehiculo)

data = data.drop(columns=['HORA_OCURRENCIA_ACC'])
data = data.drop(columns=['CLASE_DE_VEHICULO'])
data["nroinfracciones"] = data["nroinfracciones"].fillna(0)

In [6]:
data.head()

,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,GRAVEDAD,LOCALIDAD,CONDICION_ACCIDENTADO,nroinfracciones,TIPO_DE_SERVICIO,GENERO,ESTADO,HORA_OCURRENCIA_ACC_FLOAT,CLASIFICACION_VEHICULO
0,OCTUBRE,MARTES,SOLO DANOS,FONTIBON,CONDUCTOR,1.0,PUBLICO,MASCULINO,ILESO,14.666667,CUATRO_LLANTAS
1,NOVIEMBRE,VIERNES,SOLO DANOS,FONTIBON,CONDUCTOR,1.0,PUBLICO,MASCULINO,ILESO,15.333333,CUATRO_LLANTAS
2,NOVIEMBRE,VIERNES,SOLO DANOS,CIUDAD BOLIVAR,CONDUCTOR,1.0,PARTICULAR,MASCULINO,ILESO,5.166667,CUATRO_LLANTAS
3,MAYO,MARTES,SOLO DANOS,ENGATIVA,CONDUCTOR,0.0,PARTICULAR,MASCULINO,ILESO,15.166667,CUATRO_LLANTAS
4,MARZO,SABADO,SOLO DANOS,ENGATIVA,CONDUCTOR,1.0,None,MASCULINO,ILESO,13.666667,CUATRO_LLANTAS


In [7]:
data_encoders = data.copy()
label_encoders = {}
for column in data_encoders.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    data_encoders[column] = label_encoders[column].fit_transform(data_encoders[column])

X = data_encoders.drop(columns=['ESTADO'])
y = data_encoders['ESTADO']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
data_encoders.head()

,MES_OCURRENCIA_ACC,DIA_OCURRENCIA_ACC,GRAVEDAD,LOCALIDAD,CONDICION_ACCIDENTADO,nroinfracciones,TIPO_DE_SERVICIO,GENERO,ESTADO,HORA_OCURRENCIA_ACC_FLOAT,CLASIFICACION_VEHICULO
0,10,3,2,7,1,1.0,4,1,1,14.666667,0
1,9,6,2,7,1,1.0,4,1,1,15.333333,0
2,9,6,2,5,1,1.0,3,1,1,5.166667,0
3,8,3,2,6,1,0.0,3,1,1,15.166667,0
4,7,5,2,6,1,1.0,5,1,1,13.666667,0


In [9]:
# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [10]:
# Predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9430226531696725
Precision: 0.9430066659320485
Recall: 0.9430226531696725
F1 Score: 0.9430052255132586
Confusion Matrix:
[[ 66480   8352    443]
 [  7967 219010    164]
 [   329    115   1998]]


In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.88      0.89     75275
           1       0.96      0.96      0.96    227141
           2       0.77      0.82      0.79      2442

    accuracy                           0.94    304858
   macro avg       0.87      0.89      0.88    304858
weighted avg       0.94      0.94      0.94    304858



In [37]:
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = logistic_model.predict(X_test)

# Evaluar el modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86     75227
           1       0.95      0.96      0.95    227123
           2       0.83      0.21      0.34      2444

    accuracy                           0.93    304794
   macro avg       0.88      0.68      0.72    304794
weighted avg       0.92      0.93      0.92    304794

